In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm
import numpy as np
from numpy import linalg, mean, dot
from sklearn import tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split


In [18]:
with open("../collect-data/src/files/nba_data.xlsx", "rb") as file:
    nba_stats_df = pd.read_excel(file, sheet_name="NBA Stats")
    mvp_df = pd.read_excel(file, sheet_name="MVP")

In [19]:
mvp_df["MVP?"] = 1
nba_stats_df["MVP?"] = 0

In [23]:
mvp_dict = mvp_df.to_dict("index")
nba_stats_dict = nba_stats_df.to_dict("index")
mvp_rank_dict = dict()

for k in mvp_dict:
    f_name = mvp_dict[k]["FIRST NAME"]
    l_name = mvp_dict[k]["LAST NAME"]
    team = mvp_dict[k]["TEAM"]
    season = mvp_dict[k]["SEASON"]
    # Filter down dataset
    for i in list(nba_stats_dict):
        if nba_stats_dict[i]["FIRST NAME"] == f_name and nba_stats_dict[i]["LAST NAME"] == l_name and nba_stats_dict[i]["TEAM"] == team and nba_stats_dict[i]["SEASON"] == season:
            nba_stats_dict[i]["MVP?"] = 1
            mvp_rank_dict[nba_stats_dict[i]["SEASON"]] = nba_stats_dict[i]["RANK"]

for season in mvp_rank_dict:
    rank = mvp_rank_dict[season]
    if rank > 15:
        max_rank = rank
    else:
        max_rank = 15
    for j in list(nba_stats_dict):
        if nba_stats_dict[j]["SEASON"] == season and nba_stats_dict[j]["RANK"] > max_rank:
            nba_stats_dict.pop(j)

nba_stats_df = pd.DataFrame.from_dict(nba_stats_dict, orient="index")
print(nba_stats_df["MVP?"].value_counts())


MVP?
0    1020
1      27
Name: count, dtype: int64


In [21]:
hello
print(nba_stats_df[nba_stats_df["MVP?"] == "mvp"])

NameError: name 'hello' is not defined

In [ ]:
model_df = nba_stats_df[['FIRST NAME', 'LAST NAME', 'SUFFIX', 'TEAM', 'AGE', 'SEASON', 'MVP?']]

In [ ]:
nba_stats_df.head()

In [ ]:
nba_stats_df = nba_stats_df.astype({"RANK": "object"})
nba_stats_df.dtypes

In [ ]:
nba_stats_df["+/-"] = nba_stats_df["+/-"].apply(lambda x: int(x) if len(x) < 5 else 0)

In [ ]:
# Bin the rankings together
nba_stats_df["RANK"] = pd.cut(nba_stats_df["RANK"], 15)
nba_stats_df["AGE"] = pd.cut(nba_stats_df["AGE"], 5)
nba_stats_df["+/-"] = pd.cut(nba_stats_df["+/-"], 10)
print(nba_stats_df)

In [ ]:
# rank_enc = pd.get_dummies(nba_stats_df["RANK"], dtype=float)
# nba_stats_df = pd.concat([nba_stats_df, rank_enc], axis=1)

# age_enc = pd.get_dummies(nba_stats_df["AGE"], dtype=float)
# nba_stats_df = pd.concat([nba_stats_df, age_enc], axis=1)

# plus_minus_enc = pd.get_dummies(nba_stats_df["+/-"], dtype=float)
# nba_stats_df = pd.concat([nba_stats_df, plus_minus_enc], axis=1)

# season_enc = pd.get_dummies(nba_stats_df["SEASON"], dtype=float)
# nba_stats_df = pd.concat([nba_stats_df, season_enc], axis=1)

# team_enc = pd.get_dummies(nba_stats_df["TEAM"], dtype=float)
# nba_stats_df = pd.concat([nba_stats_df, team_enc], axis=1)

print(nba_stats_df)


In [ ]:
current_season_df = nba_stats_df[nba_stats_df["SEASON"] == "2023-24"]
nba_stats_df = nba_stats_df[nba_stats_df["SEASON"] != "2023-24"]
print(current_season_df.shape)
print(nba_stats_df.shape)

In [ ]:
# Change season to int
seasons_list = nba_stats_df["SEASON"].dropna().unique()
season_dict = { None: 1 }
for i in range(len(seasons_list)):
    season = seasons_list[i]
    season_dict[season] = i + 1

nba_stats_df = nba_stats_df.replace({ "SEASON": season_dict })

In [ ]:
# nba_stats_df = nba_stats_df[['AGE', 'GP', 'W', 'L', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA',
#        '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL',
#        'BLK', 'PF', 'FP', 'DD2', 'TD3', 'SEASON', 'MVP?']]
nba_stats_df = nba_stats_df.drop(columns=['RANK', 'MIN', 'PTS', 'FGM', 'FGA', 'AGE', 'FIRST NAME', 'LAST NAME', 'OREB', 'DREB',  'REB', 'SUFFIX', 'FP', 'TEAM', '+/-'], axis=1)
# nba_stats_df = nba_stats_df.drop(columns=['GP',   'W',   'L', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%',  'FTM',  'FTA',  'FT%', 'OREB', 'DREB',  'REB',  'AST',  'TOV', 'STL', 'BLK', 'PF', 'FP','DD2', 'TD3'], axis=1)
nba_stats_df.corr()

In [ ]:
plt.matshow(nba_stats_df.corr())
cb = plt.colorbar()
cb.ax.tick_params(labelsize=10)
plt.show()

In [ ]:
len(nba_stats_df)

In [ ]:
nba_stats_df.var()

In [ ]:
nba_stats_df.cov()

In [ ]:
def generate_pc(columns, variance_pct):
    scaler = MinMaxScaler()
    data_rescaled = scaler.fit_transform(columns.to_numpy())

    pca = PCA().fit(data_rescaled)
    plt.rcParams["figure.figsize"] = (35,10)

    fig, ax = plt.subplots()
    xi = np.arange(1, 20, step=1)
    y = np.cumsum(pca.explained_variance_ratio_)

    plt.ylim(0.0,1.1)
    plt.plot(xi, y, marker="o", linestyle="--", color="b")

    plt.xlabel("Number of Components")
    plt.xticks(np.arange(0, 19, step=1))
    plt.ylabel("Cumulative variance (%)")
    plt.title("The number of components needed to explain variance")

    plt.axhline(y=variance_pct, color="r", linestyle="-")
    plt.text(0.5, 0.85, f"{variance_pct * 100}% cut-off threshold", color="red", fontsize=16)

    ax.grid(axis="x")
    plt.show()
    return pca
pca = generate_pc(nba_stats_df, 0.95)

pca.fit(nba_stats_df.to_numpy())
pca.transform(nba_stats_df.to_numpy())

In [ ]:
# projected.plot(kind="scatter", x="pc1", y="pc2")
nba_stats_df.columns

In [ ]:
# Decision Tree
Y = nba_stats_df["MVP?"]
X = nba_stats_df.drop("MVP?", axis=1)
X.columns = X.columns.astype(str)
max_depths = [2,3,4,5,6,7,8,9,10]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.8, random_state=1)
for depth in max_depths:
    clf = tree.DecisionTreeClassifier(max_depth=depth)
    clf = clf.fit(X_train, Y_train)
    Y_predtest = clf.predict(X_test)
    Y_predtrain = clf.predict(X_train)
    print("depth:", depth, "test accuracy:", accuracy_score(Y_test, Y_predtest), "train accuracy:", accuracy_score(Y_train, Y_predtrain))
    print(pd.crosstab(Y_test, Y_predtest, rownames = ['Actual'], colnames =['Predicted'], margins = True))

In [ ]:
print(classification_report(Y_test, Y_predtest, zero_division=0))

In [ ]:
pd.crosstab(Y_test, Y_predtest, rownames = ['Actual'], colnames =['Predicted'], margins = True)

In [ ]:
seasons = nba_stats_df["SEASON"].unique()

for season in seasons:
    print(seasons_list[season - 1])
    season_stats_df = nba_stats_df[nba_stats_df["SEASON"] == season]
    Y = season_stats_df["MVP?"]
    X = season_stats_df.drop("MVP?", axis=1)
    X.columns = X.columns.astype(str)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.9, random_state=8)
    clf = tree.DecisionTreeClassifier(max_depth=2)
    clf = clf.fit(X_train, Y_train)
    Y_predtest = clf.predict(X_test)
    Y_predtrain = clf.predict(X_train)
    print("depth:", depth, "test accuracy:", accuracy_score(Y_test, Y_predtest), "train accuracy:", accuracy_score(Y_train, Y_predtrain))
    print(pd.crosstab(Y_test, Y_predtest, rownames = ['Actual'], colnames =['Predicted'], margins = True))
    print("\n")

In [ ]:
# Results after applying PCA
pca.fit(X_train)
explained_variance = pca.explained_variance_ratio_
cum_explained_ratio = np.cumsum(explained_variance)
optimal_num_comp = np.where(cum_explained_ratio >= 0.95)[0][0] + 1

pca = PCA(n_components=optimal_num_comp, svd_solver="full")
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

pca_tree = tree.DecisionTreeClassifier(random_state=42)
pca_tree = pca_tree.fit(X_train_pca, Y_train)

Y_pred = pca_tree.predict(X_test_pca)
print(accuracy_score(Y_test, Y_pred))

In [ ]:
print(classification_report(Y_test, Y_pred, zero_division=0))

In [ ]:
pd.crosstab(Y_test, Y_pred, rownames = ['Actual'], colnames =['Predicted'], margins = True)